In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy import sparse

# Constructing hypergraphs and corresponding comparison graphs

In [2]:
# We want the nonzero rows of W and R to sum to 1 
def row_normalize(X):
    Y = np.matrix.copy(X)
    for i in range(len(Y)):
        row = Y[i]
        row_sum = np.sum(row)
        if row_sum != 0:
            Y[i] = Y[i]/row_sum   
    return Y

In [3]:
# E: number of users
# V: number of items
# p: chance certain user watches certain movie
# e1: chance edge weight is flipped
# e2: chance user rating is flipped
# e3: chance user rating is erased
def make_hypergraphs(E, V, p, e1, e2, e3, random_seed):
    np.random.seed(random_seed)
    
    n = int(E/3)
    m = int(V/3)
    
    W = np.random.rand(V, E) # hyperedge-weight matrix, |V| x |E|, each row corresponds to a movie. 
    num_pairs = 0

    for i in range(V):
        for j in range(E):
            if W[i][j] < p:
                W[i][j] = 1

                num_pairs += 1
            else:
                W[i][j] = 0
                
    W1 = np.copy(W)
    W2 = np.copy(W)
    W3 = np.copy(W)
                
    for j in range(E):
        W1_multiplier = 100 if j < n else 1
        W2_multiplier = 100 if j < 2 * n and j >= n else 1
        W3_multiplier = 100 if j >= 2 * n else 1
       
        err = np.random.rand(1)
        if err < e1:
            W1_multiplier = 101 - W1_multiplier 
            W2_multiplier = 101 - W2_multiplier
            W3_multiplier = 101 - W3_multiplier
            
        W1[:,j] *= W1_multiplier
        W2[:,j] *= W2_multiplier
        W3[:,j] *= W3_multiplier

                
    R = np.zeros((E, V)) # edge-dependent vertex-weight matrix, |E| x |V|, each row corresponds to a user.
    true_R = np.zeros((E, V)) # true ratings

    for i in range(V):
        for j in range(E):
            if W[i][j] == 1:
                if i // n == j // m:
                    true_R[j][i] = 5
                else:
                    true_R[j][i] = 1

                err = np.random.rand(1)
                if err < e2:
                    R[j][i] = 6 - true_R[j][i]
                elif err >= e2 and err < e2 + e3:
                    W[i][j] == 0
                    # do nothing as R[j][i] is already 0
                else:
                    R[j][i] = true_R[j][i]
                    
    WG1 = np.zeros((V+E, num_pairs)) # hyperedge weight matrix, weights are ratings x 10 IF user likes category
    WG2 = np.zeros((V+E, num_pairs)) # otherwise 1
    WG3 = np.zeros((V+E, num_pairs))

    RG = np.zeros((num_pairs, V+E)) # edge-dependent vertex weight matrix, weights are 1

    curr_edge_index = 0 

    for i in range(V):
        for j in range(E):
            if R[j][i] != 0:
                # movie index = i
                # user index = V+j

                RG[curr_edge_index][V+j] = 1
                RG[curr_edge_index][i] = 1

                WG1[V+j][curr_edge_index] = W1[i][j] * R[j][i]
                WG1[i][curr_edge_index] = WG1[V+j][curr_edge_index]

                WG2[V+j][curr_edge_index] = W2[i][j] * R[j][i]
                WG2[i][curr_edge_index] = WG2[V+j][curr_edge_index]

                WG3[V+j][curr_edge_index] = W3[i][j] * R[j][i] 
                WG3[i][curr_edge_index] = WG3[V+j][curr_edge_index]

                curr_edge_index += 1
                
                
    W1s = sparse.csr_matrix(row_normalize(W1))
    W2s = sparse.csr_matrix(row_normalize(W2))
    W3s = sparse.csr_matrix(row_normalize(W3))
    Rs = sparse.csr_matrix(row_normalize(R))

    WG1s = sparse.csr_matrix(row_normalize(WG1))
    WG2s = sparse.csr_matrix(row_normalize(WG2))
    WG3s = sparse.csr_matrix(row_normalize(WG3))
    RGs = sparse.csr_matrix(row_normalize(RG))

    # create prob trans matrices
    P1 = np.transpose(W1s.dot(Rs))
    P2 = np.transpose(W2s.dot(Rs))
    P3 = np.transpose(W3s.dot(Rs))

    PG1 = np.transpose(WG1s.dot(RGs))
    PG2 = np.transpose(WG2s.dot(RGs))
    PG3 = np.transpose(WG3s.dot(RGs))

    return P1, P2, P3, PG1, PG2, PG3, R, true_R

# Computing personalized PageRank rankings

In [4]:
# given probability transition matrix P
# where P_{v,w} = Prob(w -> v)
# find pagerank scores with restart probability r
def compute_pr(P, r, n, home, eps=1e-8):
    
    x = np.ones(n) / n*1.0

    flag = True
    t=0
        
    while flag:
        x_new = (1-r)*P*x

        x_new = x_new + home * r 
        
        if np.linalg.norm(x_new - x,ord=1) < eps and t > 100:
            flag = False
        t=t+1
        x = x_new
    
    return x

In [5]:
def get_rankings(E, V, R, P1, P2, P3, PG1, PG2, PG3, r):
    
    rankings_hg = np.zeros((E, V)) # each row corresponds to a user. 
    rankings_g = np.zeros((E, V)) # each row corresponds to a user. 
    n = int(E/3)
    
    for i in range(E):

        if i < n:
            P = P1
            PG = PG1
        elif i < 2 * n:
            P = P2
            PG = PG2
        else: 
            P = P3
            PG = PG3

        # personalize the algorithm by restarting at any of the movies a certain user originally watched
        home_hg = np.zeros(V)

        for j in range(V):
            if R[i][j] != 0:
                home_hg[j] = 1

        if np.sum(home_hg) > 0:
            home_hg = home_hg / np.sum(home_hg)

        rankings_hg[i,:] = compute_pr(P, r, V, home_hg).flatten()

        # same process for the graph
        home_g = np.zeros(V+E)
        home_g[V+i] = 1

        curr_rankings_g = compute_pr(PG, r, V+E, home_g).flatten()
        rankings_g[i,:] = curr_rankings_g[:V]
        
    return rankings_hg, rankings_g

# Evaluating rankings

In [6]:
# Source: https://www.aaai.org/Papers/IJCAI/2007/IJCAI07-444.pdf
def calc_avg_doa(num_users, num_movies, ratings, rankings):
    
    n = num_users/3
    m = num_movies/3
    
    total_pairs = 0
    correct_pairs = 0
    
    # All pairs of movies. 
    for i in range(num_movies):
        for j in range(i+1, num_movies):
            for user in range(num_users):

                if i // m != j // m:
                    if user // n == i // m:
                        total_pairs += 1
                        if rankings[user][i] > rankings[user][j]:
                            correct_pairs += 1
                    elif user // n == j // m:
                        total_pairs += 1
                        if rankings[user][i] < rankings[user][j]:
                            correct_pairs += 1
       
    if total_pairs == 0:
        return -1
    return correct_pairs/total_pairs

def calc_avg_udoa(num_users, num_movies, ratings, rankings):
    
    n = num_users/3
    m = num_movies/3
    
    total_pairs = 0
    correct_pairs = 0
    
    # All pairs of movies. 
    for i in range(num_movies):
        for j in range(i+1, num_movies):
            for user in range(num_users):
                
                if ratings[user][i] == 0 and ratings[user][j] == 0:
                    if i // m != j // m:
                        if user // n == i // m:
                            total_pairs += 1
                            if rankings[user][i] > rankings[user][j]:
                                correct_pairs += 1
                        elif user // n == j // m:
                            total_pairs += 1
                            if rankings[user][i] < rankings[user][j]:
                                correct_pairs += 1
       
    if total_pairs == 0:
        return -1
    return correct_pairs/total_pairs

In [7]:
def do_everything(E, V, p, e1, e2, e3, random_seed):
    
    n = int(E/3)
    m = int(V/3)
    
    P1, P2, P3, PG1, PG2, PG3, R, true_R = make_hypergraphs(E, V, p, e1, e2, e3, random_seed)


    rankings_hg, rankings_g = get_rankings(E, V, R, P1, P2, P3, PG1, PG2, PG3, 0.15)
    
    avgdoa_hg = calc_avg_doa(E, V, true_R, rankings_hg)
    avgdoa_g = calc_avg_doa(E, V, true_R, rankings_g)
    avgudoa_hg = calc_avg_udoa(E, V, true_R, rankings_hg)
    avgudoa_g = calc_avg_udoa(E, V, true_R, rankings_g)

    return avgdoa_hg, avgdoa_g, avgudoa_hg, avgudoa_g

In [8]:
# To remove the effects of randomness, average results generated by many random seeds. 
def do_everything_n_times(E, V, p, e1, e2, e3, n):
    print("n=%d, p=%.3f, e1=%.3f, e2=%.3f, e3=%.3f" % (n, p, e1, e2, e3))
    
    if n == 0:
        return
    
    d_hg = 0
    d_g = 0
    ud_hg = 0
    ud_g = 0

    
    for i in range(n):
        d1, d2, ud1, ud2 = do_everything(E, V, p, e1, e2, e3, i)
        d_hg += d1
        d_g += d2
        ud_hg += ud1
        ud_g += ud2
        
    d_hg = d_hg/n
    d_g = d_g/n
    ud_hg = ud_hg/n
    ud_g = ud_g/n
    
    return d_hg, d_g, ud_hg, ud_g

# Running

In [9]:
n = 50
m = 50
    
E = 3 * n # number of "users"
V = 3 * m # number of "movies

In [10]:
probs = np.linspace(0, 1, num=11)
num_probs = len(probs)

e1_hg = np.zeros(num_probs)
e1_g = np.zeros(num_probs)
e2_hg = np.zeros(num_probs)
e2_g = np.zeros(num_probs)
e3_hg = np.zeros(num_probs)
e3_g = np.zeros(num_probs)

e1_hg_u = np.zeros(num_probs)
e1_g_u = np.zeros(num_probs)
e2_hg_u = np.zeros(num_probs)
e2_g_u = np.zeros(num_probs)
e3_hg_u = np.zeros(num_probs)
e3_g_u = np.zeros(num_probs)

In [11]:
d_hg, d_g, ud_hg, ud_g = do_everything_n_times(E, V, 0.15, 0, 0, 0, 10)

e1_hg[0] = d_hg
e2_hg[0] = d_hg
e3_hg[0] = d_hg

e1_g[0] = d_g
e2_g[0] = d_g
e3_g[0] = d_g

e1_hg_u[0] = ud_hg
e2_hg_u[0] = ud_hg
e3_hg_u[0] = ud_hg

e1_g_u[0] = ud_g
e2_g_u[0] = ud_g
e3_g_u[0] = ud_g

n=10, p=0.150, e1=0.000, e2=0.000, e3=0.000


In [12]:
for i in range(1, num_probs):
    prob = probs[i]
    
    e1hg, e1g, e1hgu, e1gu = do_everything_n_times(E, V, 0.15, prob, 0, 0, 10)
    
    e1_hg[i] = e1hg
    e1_g[i] = e1g
    
    e1_hg_u[i] = e1hgu
    e1_g_u[i] = e1gu
    
    e2hg, e2g, e2hgu, e2gu = do_everything_n_times(E, V, 0.15, 0, prob, 0, 10)
    
    e2_hg[i] = e2hg
    e2_g[i] = e2g
    
    e2_hg_u[i] = e2hgu
    e2_g_u[i] = e2gu
    
    e3hg, e3g, e3hgu, e3gu = do_everything_n_times(E, V, 0.15, 0, 0, prob, 10)
    
    e3_hg[i] = e3hg
    e3_g[i] = e3g
    
    e3_hg_u[i] = e3hgu
    e3_g_u[i] = e3gu
    
    print()

n=10, p=0.150, e1=0.100, e2=0.000, e3=0.000
n=10, p=0.150, e1=0.000, e2=0.100, e3=0.000
n=10, p=0.150, e1=0.000, e2=0.000, e3=0.100

n=10, p=0.150, e1=0.200, e2=0.000, e3=0.000
n=10, p=0.150, e1=0.000, e2=0.200, e3=0.000
n=10, p=0.150, e1=0.000, e2=0.000, e3=0.200

n=10, p=0.150, e1=0.300, e2=0.000, e3=0.000
n=10, p=0.150, e1=0.000, e2=0.300, e3=0.000
n=10, p=0.150, e1=0.000, e2=0.000, e3=0.300

n=10, p=0.150, e1=0.400, e2=0.000, e3=0.000
n=10, p=0.150, e1=0.000, e2=0.400, e3=0.000
n=10, p=0.150, e1=0.000, e2=0.000, e3=0.400

n=10, p=0.150, e1=0.500, e2=0.000, e3=0.000
n=10, p=0.150, e1=0.000, e2=0.500, e3=0.000
n=10, p=0.150, e1=0.000, e2=0.000, e3=0.500

n=10, p=0.150, e1=0.600, e2=0.000, e3=0.000
n=10, p=0.150, e1=0.000, e2=0.600, e3=0.000
n=10, p=0.150, e1=0.000, e2=0.000, e3=0.600

n=10, p=0.150, e1=0.700, e2=0.000, e3=0.000
n=10, p=0.150, e1=0.000, e2=0.700, e3=0.000
n=10, p=0.150, e1=0.000, e2=0.000, e3=0.700

n=10, p=0.150, e1=0.800, e2=0.000, e3=0.000
n=10, p=0.150, e1=0.000, 

In [22]:
print(repr(e1_hg))
print(repr(e2_hg))
print(repr(e3_hg))

array([0.94940453, 0.89848693, 0.82704067, 0.74324467, 0.62540733,
       0.49242613, 0.36750067, 0.26243267, 0.18223187, 0.13549627,
       0.10881147])
array([0.94940453, 0.90492933, 0.84245187, 0.75246173, 0.6328692 ,
       0.50107053, 0.3698528 , 0.2556104 , 0.1741616 , 0.12265107,
       0.0921452 ])
array([0.94940453, 0.9404888 , 0.93150493, 0.922684  , 0.9099368 ,
       0.89168893, 0.8604868 , 0.80022707, 0.69785333, 0.53871467,
       0.        ])


In [23]:
print(repr(e1_hg_u))
print(repr(e2_hg_u))
print(repr(e3_hg_u))

array([0.9945177 , 0.9662201 , 0.90067333, 0.80509172, 0.65915388,
       0.48615968, 0.32233435, 0.1860325 , 0.08533305, 0.03223491,
       0.00824078])
array([0.9945177 , 0.97265209, 0.92024541, 0.82033433, 0.67180449,
       0.50135154, 0.33004967, 0.18215345, 0.08144394, 0.02601919,
       0.00355345])
array([0.9945177 , 0.99160386, 0.98671406, 0.97918676, 0.96451392,
       0.93961596, 0.89910315, 0.82580012, 0.70959501, 0.53975818,
       0.        ])


In [24]:
print(repr(e1_g))
print(repr(e2_g))
print(repr(e3_g))

array([0.877446  , 0.85307467, 0.8157572 , 0.76803493, 0.69882947,
       0.61151493, 0.5110576 , 0.40563053, 0.2995676 , 0.2154764 ,
       0.15916573])
array([0.877446  , 0.8507576 , 0.8032096 , 0.72592107, 0.6190004 ,
       0.5016388 , 0.38112947, 0.2744768 , 0.19630067, 0.14760333,
       0.12108453])
array([0.877446  , 0.88087853, 0.8834332 , 0.88397053, 0.87412773,
       0.84958547, 0.80424573, 0.72571493, 0.61913453, 0.51426307,
       0.        ])


In [25]:
print(repr(e1_g_u))
print(repr(e2_g_u))
print(repr(e3_g_u))

array([0.99270582, 0.96751921, 0.91912597, 0.85440711, 0.75933224,
       0.63848409, 0.49910307, 0.35271921, 0.20560955, 0.08893313,
       0.01083144])
array([0.99270582, 0.965318  , 0.90493743, 0.80225218, 0.65915387,
       0.50204616, 0.34071286, 0.19799605, 0.09427907, 0.03231989,
       0.0049148 ])
array([0.99270582, 0.98899914, 0.98204131, 0.97087387, 0.94664799,
       0.90568451, 0.84300435, 0.74679472, 0.6263997 , 0.51489551,
       0.        ])
